In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data_utils import BasicFunctions, PriceFunctions
import math
import tensorflow as tf

%matplotlib inline

c:\programdata\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
bf = BasicFunctions()
pf = PriceFunctions()

In [3]:
mean, std, pd_Xtrain, pd_ytrain, pd_Xtest, pd_ytest, Xtrain, ytrainOrig, Xtest, ytestOrig = pf.return_data()

In [4]:
ytrain = bf.convert_to_one_hot(ytrainOrig, 9)

In [5]:
ytest = bf.convert_to_one_hot(ytestOrig, 9)

In [6]:
pd_Xtrain

,Open,Close,High,Low,Volume
Time,,,,,
1364774400,93.25000,93.10000,100.00000,93.03000,345.583889
1364778000,93.37999,93.10000,93.48797,93.10000,45.243335
1364781600,93.17000,93.74000,94.00000,93.10999,466.311420
1364785200,93.80000,92.70002,93.80000,92.49999,96.316180
1364788800,93.15501,93.15503,93.15503,93.15501,5.000000
1364792400,93.70000,93.79000,93.79000,93.70000,22.090000
1364796000,94.06751,94.00001,94.48000,94.00000,84.000000
1364799600,94.06751,94.00001,94.48000,94.00000,84.000000
1364803200,93.55000,94.00000,94.00000,93.55000,3.807371
